In [2]:
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [3]:
project = 'taxi-project-461423'
location = 'EU'
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [4]:
current_year = datetime.now().year
current_year

2025

In [5]:
# Function to execute a BigQuery query and return a DataFrame

def query_to_dataframe(query: str) -> pd.DataFrame:
    """
    Executes a SQL query in BigQuery and returns a Pandas DataFrame.

    Parameters:
    - query (str): The SQL query to execute.

    Return:
    - pd.DataFrame : The DataFrame containing the results of the query.
    """
    try:
        df = client.query(query).to_dataframe()
        print(f"Query executed successfully. Retrieved {df.shape[0]} rows.")
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()

## Question 10: How often do passengers tip, and what factors (time of day, borough, fare amount) influence tip amounts?

In [6]:
query_tipping_behavior_analysis = """
SELECT *
FROM `taxi-project-461423.views_fordashboard.tipping_behavior_analysis`
"""
tipping_behavior_analysis_df = query_to_dataframe(query_tipping_behavior_analysis)
tipping_behavior_analysis_df.head()

Query executed successfully. Retrieved 589712 rows.


,trip_date,year,month,pickup_hour,pickup_borough,dropoff_borough,total_trips,tipped_trips,tip_frequency_percentage,avg_tip_amount,avg_total_fare,avg_fare,avg_tip_percentage
0,2023-03-25,2023,3,14,Manhattan,Manhattan,4558,4406,96.67,3.11,20.42,13.34,15.04
1,2023-03-02,2023,3,22,Manhattan,Manhattan,4843,4709,97.23,3.31,21.99,13.71,14.83
2,2023-03-06,2023,3,12,Unknown,Unknown,25,23,92.00,4.62,35.72,26.94,14.90
3,2023-03-08,2023,3,8,Manhattan,Manhattan,4213,4085,96.96,3.07,20.63,13.59,14.76
4,2023-03-18,2023,3,1,Manhattan,Manhattan,3811,3551,93.18,3.13,22.00,13.88,13.94


In [7]:
# filter rows where the year is between 2022 and the current year (inclusive)
filtered_tipping_behavior_analysis_df = tipping_behavior_analysis_df[(tipping_behavior_analysis_df['year'] >= 2022) & (tipping_behavior_analysis_df['year'] <= current_year)]
filtered_tipping_behavior_analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 589588 entries, 0 to 589711
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   trip_date                 589588 non-null  dbdate 
 1   year                      589588 non-null  Int64  
 2   month                     589588 non-null  Int64  
 3   pickup_hour               589588 non-null  Int64  
 4   pickup_borough            589588 non-null  object 
 5   dropoff_borough           589588 non-null  object 
 6   total_trips               589588 non-null  Int64  
 7   tipped_trips              589588 non-null  Int64  
 8   tip_frequency_percentage  589588 non-null  float64
 9   avg_tip_amount            589588 non-null  float64
 10  avg_total_fare            589588 non-null  float64
 11  avg_fare                  589588 non-null  float64
 12  avg_tip_percentage        589588 non-null  float64
dtypes: Int64(5), dbdate(1), float64(5), object(2)
mem

In [8]:
# Aggregate data by month to reduce noise
filtered_tipping_behavior_analysis_df["trip_date"] = pd.to_datetime(filtered_tipping_behavior_analysis_df["trip_date"])
df_monthly = filtered_tipping_behavior_analysis_df.groupby(pd.Grouper(key="trip_date", freq="M"))["tip_frequency_percentage"].mean().reset_index()
df_monthly.head()

/tmp/ipython-input-8-2639585145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tipping_behavior_analysis_df["trip_date"] = pd.to_datetime(filtered_tipping_behavior_analysis_df["trip_date"])
/tmp/ipython-input-8-2639585145.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = filtered_tipping_behavior_analysis_df.groupby(pd.Grouper(key="trip_date", freq="M"))["tip_frequency_percentage"].mean().reset_index()


,trip_date,tip_frequency_percentage
0,2022-01-31,80.188295
1,2022-02-28,77.925699
2,2022-03-31,79.288799
3,2022-04-30,81.181378
4,2022-05-31,81.512458


In [9]:
# Create a smoother line plot
fig = px.line(
    df_monthly,
    x="trip_date",
    y="tip_frequency_percentage",
    title="Tip Frequency Over Time (Monthly Average)",
    labels={"tip_frequency_percentage": "Tip Frequency (%)", "trip_date": "Date"},
    template="plotly_white",
    line_shape="spline"  # Makes the line smoother
)

fig.show()


In [10]:
fig = px.bar(
    filtered_tipping_behavior_analysis_df.groupby("pickup_borough")["avg_tip_percentage"].mean().reset_index(),
    x="pickup_borough",
    y="avg_tip_percentage",
    title="Average Tip Percentage by Pickup Borough",
    labels={"avg_tip_percentage": "Average Tip (%)"},
    template="plotly_white",
    color="avg_tip_percentage",
    color_continuous_scale="Blues"
)
fig.show()

In [11]:
heatmap_data = filtered_tipping_behavior_analysis_df.groupby(["pickup_hour", "pickup_borough"])["avg_tip_percentage"].mean().unstack()

fig = go.Figure(
    data=go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale="YlGnBu"
    )
)

fig.update_layout(
    title="Tipping Trends by Time of Day & Borough",
    xaxis_title="Borough",
    yaxis_title="Hour of Day",
    template="plotly_white"
)
fig.show()

## Question 11: How much revenue is generated from additional charges (MTA tax, congestion surcharge, airport fees), and has it changed over time?

In [12]:
query_additional_charges_revenue = """
SELECT *
FROM `taxi-project-461423.views_fordashboard.additional_charges_revenue`
"""
additional_charges_revenue_df = query_to_dataframe(query_additional_charges_revenue)
additional_charges_revenue_df.head()

Query executed successfully. Retrieved 1205 rows.


,trip_date,year,month,total_trips,total_MTA_tax,total_congestion_surcharge,total_airport_fees,total_additional_revenue,avg_additional_charge_per_trip
0,2023-08-04,2023,8,85706,42512.28,197037.5,14126.00,253675.78,2.96
1,2023-08-12,2023,8,83558,41466.60,194125.0,12202.75,247794.35,2.97
2,2023-08-07,2023,8,77323,38356.20,175215.0,15176.00,228747.20,2.96
3,2024-03-18,2024,3,85573,42446.00,195705.0,17542.00,255693.00,2.99
4,2023-05-04,2023,5,118952,59144.05,279022.5,14859.25,353025.80,2.97


In [13]:
# filter rows where the year is between 2022 and the current year (inclusive)
filtered_additional_charges_revenue_df = additional_charges_revenue_df[(additional_charges_revenue_df['year'] >= 2022) & (additional_charges_revenue_df['year'] <= current_year)]
filtered_additional_charges_revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1187 entries, 0 to 1203
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   trip_date                       1187 non-null   dbdate 
 1   year                            1187 non-null   Int64  
 2   month                           1187 non-null   Int64  
 3   total_trips                     1187 non-null   Int64  
 4   total_MTA_tax                   1187 non-null   float64
 5   total_congestion_surcharge      1187 non-null   float64
 6   total_airport_fees              1187 non-null   float64
 7   total_additional_revenue        1187 non-null   float64
 8   avg_additional_charge_per_trip  1187 non-null   float64
dtypes: Int64(3), dbdate(1), float64(5)
memory usage: 96.2 KB


In [14]:
# Convert trip_date to datetime
filtered_additional_charges_revenue_df["trip_date"] = pd.to_datetime(filtered_additional_charges_revenue_df["trip_date"])

# Fill missing values with 0 (assumes missing airport fees mean no charge for that trip)
filtered_additional_charges_revenue_df.fillna(0, inplace=True)

# Create stacked area plot
fig = px.area(
    filtered_additional_charges_revenue_df,
    x="trip_date",
    y=["total_MTA_tax", "total_congestion_surcharge", "total_airport_fees"],
    labels={"trip_date": "Date", "value": "Revenue ($)", "variable": "Charge Type"},
    title="Revenue from Additional Charges Over Time",
    template="plotly_white"
)

fig.show()

/tmp/ipython-input-14-1416005986.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipython-input-14-1416005986.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

